In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
import json
import os

In [ ]:
with open("./config.json", "r") as fp:
    config = json.load(fp)

In [ ]:
DATA_DIR_PATH = config["data_dir_path"]
SNLI_DIR_PATH = os.path.join(DATA_DIR_PATH, "snli_1.0")

# Read SNLI datasets

In [ ]:
df_snli_train = pd.read_csv(os.path.join(SNLI_DIR_PATH, "snli_1.0_train.txt"), delimiter='\t')
df_snli_dev = pd.read_csv(os.path.join(SNLI_DIR_PATH, "snli_1.0_dev.txt"), delimiter='\t')
df_snli_test = pd.read_csv(os.path.join(SNLI_DIR_PATH, "snli_1.0_test.txt"), delimiter='\t')

In [ ]:
df_snli_train = df_snli_train[["gold_label", "sentence1_binary_parse", "sentence2_binary_parse", "sentence1", "sentence2"]].query("gold_label != '-'").dropna(how="any")
df_snli_dev = df_snli_dev[["gold_label", "sentence1_binary_parse", "sentence2_binary_parse", "sentence1", "sentence2"]].query("gold_label != '-'").dropna(how="any")
df_snli_test = df_snli_test[["gold_label", "sentence1_binary_parse", "sentence2_binary_parse", "sentence1", "sentence2"]].query("gold_label != '-'").dropna(how="any")

In [ ]:
df_snli_train.head()

In [ ]:
df_snli_dev.head()

In [ ]:
df_snli_test.head()

# Generate tokens from binary_parse and concat to df

In [ ]:
import re

def convert_binary_parse_to_tokens(binary_parse):
    tokens = binary_parse
    
    tokens = re.sub("\(", "", tokens) # remove left parentheses
    tokens = re.sub("\)", "", tokens) # remove right parentheses
    tokens = re.sub("\s+", " ", tokens) # remove multi space
    tokens = re.sub("``", "\"", tokens) # replace `` to " 
    tokens = re.sub("''", "\"", tokens) # replace '' to "
    tokens = tokens.strip()
    return tokens

def concat_tokens_to_df(df):
    tokens1 = []
    tokens2 = []

    for sample in list(df.itertuples()):
        tokens1.append(convert_binary_parse_to_tokens(sample.sentence1_binary_parse))
        tokens2.append(convert_binary_parse_to_tokens(sample.sentence2_binary_parse))
    
    df["tokens1"] = tokens1
    df["tokens2"] = tokens2

In [ ]:
concat_tokens_to_df(df_snli_train)
df_snli_train.head()

In [ ]:
concat_tokens_to_df(df_snli_dev)
df_snli_dev.head()

In [ ]:
concat_tokens_to_df(df_snli_test)
df_snli_test.head()

In [ ]:
df_snli_train.to_csv(os.path.join(DATA_DIR_PATH, "snli_train.tsv"), sep="\t")
df_snli_dev.to_csv(os.path.join(DATA_DIR_PATH, "snli_dev.tsv"), sep="\t")
df_snli_test.to_csv(os.path.join(DATA_DIR_PATH, "snli_test.tsv"), sep="\t")